<a href="https://colab.research.google.com/github/rodrigorenemenegazzo/Artificial-Intelligence/blob/main/Algoritmos_Geneticos_Caixeiro_viajante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 17:20:43 2022

@author: Rodrigo Rene Menegazzo
"""

#https://www.tads.ufpr.br/pluginfile.php/18884/mod_resource/content/0/03_algoritmos_geneticos.pdf
"""
descubra o percurso de menor distância que passe
uma única vez por todas as cidades e retorne à cidade de origem.

"""

#sorteio das cidades (representada por x e y, em um espaço de 
# 100 por 100 pixels. 

import matplotlib.pyplot as plt
import numpy as np
import collections
import random


def plot_cidades(cidades, km=0):
    x=[]
    y=[]
    for cidade in range(len(cidades)):
        temp_x, temp_y = (cidades[cidade,:])
        x.append(temp_x)
        y.append(temp_y)
    x.append(x[0])
    y.append(y[0])
    #Plot
    fig, ax = plt.subplots()
    #Print o total da distancia do individuo (caminho) plotado
       
    if km != 0:
        #Add blue lines
        ax.plot(x, y, label=km)
        #Add Cidades
        ax.plot(x, y, 'or', label='Cidades')
        #Add first elemnt (Grenn) 
        ax.plot(x[0], y[0], 'og', label='Inicio')
        #Add last element (Yellow)
        ax.plot(x[-2], y[-2], 'oy', label='Fim')
        
    #Add only Cidades
    else:
        ax.plot(x, y, 'or', label='Cidades')
    ax.legend(bbox_to_anchor=(1.25, 1), loc='upper right', borderaxespad=0.)        

    plt.show()    


def one_distance(p1, p2):
    """Find the distance between points p1 and p2"""
    return np.sqrt(np.sum(np.power(p2 - p1, 2))) 

#calcula a soma da distancia de cada caminho gerado com relacao
# as cidades existentes
def distance(caminho):  #0 à 99 caminho
    soma = 0 
    for i in range(len(caminho)):
        if i < (len(caminho)-1):
            p1 = caminho[i]
            p2 = caminho[i+1]
            dist = one_distance(p1, p2)
        else: #calcula a distancia do retorno ao ponto inicial
            p1 = caminho[i]
            p2 = caminho[0]
            dist = one_distance(p1, p2)
        soma += dist
    return(int(soma))
    
#Algoritmos genéticos-------------
def pop_inicial():
    list = []
    for x in range(0,11): #numero de individuos
       #range(100)= numeros de 0 à 99, 100 numeros 
       list.append(np.random.choice(range(100), 100, replace=False).tolist())
    return(list)


def crossover(pai1, pai2):

    #Posicionamento do crossover, sorteio aleatorio
    ponto_corte = random.randint(1,int(len(pai1)-2))
   
    #Crossover
    filho = []
    for g in range(0, ponto_corte):
        filho.append(pai1[g])
    
    for x in range(len(pai2)):
        if pai2[x] not in filho: 
            filho.append(pai2[x])
       
    return(filho)


def mutacao(pop):
    #sorteia um dos 10 novos individuos e realiza mutacao aleatoria
    city = random.randint(0,len(pop)-1)
    #taxa de mutação: 1%
    #Na mesma cidade sorteio duas cidades aleatorias e troca de posição 
    p1 = random.randint(0,99)    
    p2 = random.randint(0,99)
    tmp = pop[city][p1]
    pop[city][p1] = pop[city][p2]
    pop[city][p2] = tmp
    
    return(pop)   


def evolucao(selecionados):
    
    lista = []    
    #Embaralhar os individuos antes do crossover
    random.shuffle(selecionados)
     
    #acesso a cada individuo para executar Crossover
    for i in range(len(selecionados)):
        if i < (len(selecionados)-1):
            pai1 = selecionados[i]
            pai2 = selecionados[i+1]
            #Crossover
            lista.append(crossover(pai1, pai2))
            lista.append(crossover(pai2, pai1))   
        else:
            pai1 = selecionados[i]
            pai2 = selecionados[0]
            #Crossover
            lista.append(crossover(pai1, pai2))
            lista.append(crossover(pai2, pai1)) 
    
    new_pop = lista 
    #mutacao    
    
    return(new_pop)


def selecao(dictionary):
    sorted_dict = collections.OrderedDict(sorted(dictionary.items()))
    #Retorna com 50% dos melhores individuos
    best50 = list(sorted_dict.keys())[:5]
    b50_list_invividuos = []
    for i in range(len(best50)):
        b50_list_invividuos.append(sorted_dict[best50[i]])
        
    return(b50_list_invividuos, best50[0])

def geracao(populacao):
    dic = {}
    for i in populacao:
        distancia = distance(i)
        #monta o dicionario com os valores das somas das distancias
        #e o respectivo caminho(um individuo), já ordenado.
        dic[distancia] = i 
    selected, km = selecao(dic)
    new_pop = evolucao(selected)
    new_pop = mutacao(new_pop)
    
    return(new_pop, selected[0], km)   

#------------------------------------------
#"""
random.seed(123)
np.random.seed(123)

##Gerando cidades
cidades = np.random.randint(1,100, size=(100, 2))
#Plotando somente as cidades
plot_cidades(cidades)

#Gerando população inicial
pop = pop_inicial()

#Loop das geracoes
ger=0
while ger <= 2000:
    print("Geracao: ",ger)
    pop, best, km = geracao(pop)
    #Plota caminho(indivíduo)
    plot_cidades(cidades[best], km) 
    ger += 1

"""
#------------------------------------------   
   #  TESTE -----
    
random.seed(123)
np.random.seed(123)
##Gerando cidades
cidades = np.random.randint(1,100, size=(100, 2))
#Plotando somente as cidades
plot_cidades(cidades)
#Gerando população inicial
pop = pop_inicial()


pop, best, km = geracao(pop)
#Plota caminho(indivíduo)
plot_cidades(cidades[best], km) 



pai1 = pop[1]
pai2 = pop[0]

#crossover(pai1, pai2)

#Posicionamento do crossover, sorteio aleatorio
ponto_corte = random.randint(1,int(len(pai1)-2))
porcentagem_dopai = int((len(pai1)*porcentagem))

#Crossover
filho = []
for g in range(0, ponto_corte):
    filho.append(pai1[g])

for x in range(len(pai2)):
    if pai2[x] not in filho: 
        filho.append(pai2[x])





#return(filho)



for g in range(len(pai2)):
    #pai2(inicio) p/ filho
    if g <= pontoA and pontoA != 0:
        if pai2[g] not in corePai1 and pai2[g] not in filho:
            print("upou na lista primeiro if: ", g, "n: ", pai2[g])
            filho[g] = pai2[g]
        else:


 
#CROSSOVER   

filho = ["0"] * 100



for g in range(len(pai2)):
    #pai2(inicio) p/ filho
    if g <= pontoA and pontoA != 0:
        if pai2[g] not in corePai1 and pai2[g] not in filho:
            print("upou na lista primeiro if: ", g, "n: ", pai2[g])
            filho[g] = pai2[g]
        else:
            x = pai2[g]    
            while x in corePai1 or x in filho:
                if x in filho:
                    x = pai2[random.randrange(0, len(pai2), 1)]
                    print("Problema no IF: ", x, "g: ",g)
                else:
                    print("Problema no ELSE: ", x, "g: ",g)
                    for c in range(len(corePai1)):
                        if x == corePai1[c]:
                            x = corePai2[c]
            filho[g] = x
            print("upou na lista primeiro if, While g: ", g, "n:", x)

    #pai1(core) p/ filho
    elif g > pontoA and g <= pontoB or g == 0:  
        print("executou o elif g: ", g)
        filho[g] = pai1[g]  

    #pai2(fim) p/ filho
    if g > pontoB:
        print("terceiro if")
        if pai2[g] not in corePai1:
            filho[g] = pai2[g]                        
        else:
            x = pai2[g]    
            while x in corePai1:
                for c in range(len(corePai1)):
                    if x == corePai1[c]:
                        x = corePai2[c]
            filho[g] = x
    
print(filho)

dictio = {}
for i in range(len(filho)):
    dictio[filho[i]]= i
if len(dictio) == 100:
    print("OK 100!")
else:
    print("False")

#"""
 
    
#OLDDDDDDDDDDDDDDDDDDD
"""

 #Posicionamento do crossover, sorteio aleatorio
xx = random.randint(0,2)
porcentagem_dopai = int((len(pai1)*porcentagem))

if xx == 0:  #Center
    center = int((len(pai1)/2))
    pontoA = int(center - (porcentagem_dopai/2))
    pontoB = int(center + (porcentagem_dopai/2))
if xx == 1:  #Left
    pontoA = 0
    pontoB = porcentagem_dopai      
    
if xx == 2:  #Right
    pontoA = len(pai1) -porcentagem_dopai
    pontoB = len(pai1)


#Gerando core dos pais
corePai1=[]
corePai2=[]
for pp in range(pontoA, pontoB):
    corePai1.append(pai1[pp])
    corePai2.append(pai2[pp])
 
#CROSSOVER   
filho = ["0"] * 100
for g in range(len(pai2)):
    #pai2(inicio) p/ filho
    if g <= pontoA and pontoA != 0:
        if pai2[g] not in corePai1 and pai2[g] not in filho:
            print("upou na lista primeiro if: ", g, "n: ", pai2[g])
            filho[g] = pai2[g]
        else:
            x = pai2[g]    
            while x in corePai1 or x in filho:
                if x in filho:
                    x = pai2[random.randrange(0, len(pai2), 1)]
                    print("Problema no IF: ", x, "g: ",g)
                else:
                    print("Problema no ELSE: ", x, "g: ",g)
                    for c in range(len(corePai1)):
                        if x == corePai1[c]:
                            x = corePai2[c]
            filho[g] = x
            print("upou na lista primeiro if, While g: ", g, "n:", x)

    #pai1(core) p/ filho
    elif g > pontoA and g <= pontoB or g == 0:  
        print("executou o elif g: ", g)
        filho[g] = pai1[g]  

    #pai2(fim) p/ filho
    if g > pontoB:
        print("terceiro if")
        if pai2[g] not in corePai1:
            filho[g] = pai2[g]                        
        else:
            x = pai2[g]    
            while x in corePai1:
                for c in range(len(corePai1)):
                    if x == corePai1[c]:
                        x = corePai2[c]
            filho[g] = x
    
print(filho)

dictio = {}
for i in range(len(filho)):
    dictio[filho[i]]= i
if len(dictio) == 100:
    print("OK 100!")
else:
    print("False")    
    
    """
    